# Loading and Viewing BubbleML

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import numba as nb

### Create an h5py file
In this case, we are using the simulation run with a wall temperature of 100 degrees Celsius.
The simulation file includes a list of keys:
  1. `dfun` is a signed distance function from the bubbble interface
  2. `pressure` is the pressure gradient
  3. `temperature` is the temperature map
  4. `velx` is the velocity in the x direction
  5. `vely` is the velocity in the y direction
  6. `x` and `y `are coordinate grids
  7. `int/real-runtime-params` are metadata associated with the simulation run.
     This includes things like the Reynold's number, simulation dimensions, etc.

In [ ]:
twall_100 = h5py.File('Twall-100.hdf5', 'r')
for idx, key in enumerate(twall_100.keys()):
    print(f'{idx + 1}. {key}')

### Tensor sizes
Each tensor is laid out [T x X x Y] (time, x-direction, y-direction). In this case, there are 200 time steps and the domain resolution is 48x48

In [ ]:
[time_res, x_res, y_res] = twall_100['temperature'][:].shape

print(f'# Timesteps: {time_res}')
print(f'Domain resolution: {x_res} x {y_res}')

### Visualizing the different fields
The data can be easily loaded into numpy (or torch, tensorflow, etc) arrays and visualized with matplotlib.

In [ ]:
temp = twall_100['temperature'][:]
velx = twall_100['velx'][:]
vely = twall_100['vely'][:]
pres = twall_100['pressure'][:]
dfun = twall_100['dfun'][:]

# compute the velocity magnitude
mag = np.sqrt(velx**2 + vely**2)

# plot the 50-th timestep for each variable
fig, ax = plt.subplots(1, 4, figsize=(12, 5))

data = {
    'Temperature': temp[50],
    'Velocity Mag.': mag[50],
    'Pressure Grad.': pres[50],
    'Distance Func.': dfun[50]
}

for idx, (key, im) in enumerate(data.items()):
    im = ax[idx].imshow(np.flipud(im))
    fig.colorbar(im, ax=ax[idx], shrink=0.5)
    ax[idx].set_title(key)
    ax[idx].set_xticks([])
    ax[idx].set_yticks([])

### Visualizing different timesteps
By progressively indexing along the time axis (dimension 0), we are able to see the progression as it leaves the heater surface.

In [ ]:
temp = twall_100['temperature'][:]
mag = np.sqrt(twall_100['velx'][:]**2 + twall_100['vely'][:]**2)

timesteps = range(40, 52, 2)

fig, ax = plt.subplots(2, len(timesteps), figsize=(15, 5))

for idx, step in enumerate(timesteps):
    ax[0, idx].imshow(np.flipud(temp[step]))
    ax[1, idx].imshow(np.flipud(mag[step]))
    for row in range(2):
        ax[row, idx].set_title(f'Step {step}')
        ax[row, idx].set_xticks([])
        ax[row, idx].set_yticks([])
    ax[0,0].set_ylabel('Temperature')
    ax[1,0].set_ylabel('Velocity Mag.')

### Using dfun
dfun is a *signed distance function* to the liquid-vapor interfaces. 
dfun > 0 means the point is in vapor, dfun < 0 means the point is in liquid. It is also a convenient way to identify the bubble interfaces. 

The function `get_interface_mask` matches the function used in the simulations. [See equation (8)](https://doi.org/10.1016/j.ijmultiphaseflow.2019.103099). We use numba to jit compile this function for performance.

In [ ]:
@nb.njit
def get_interface_mask(dgrid):
    r""" heavy-side function to determine the bubble interfaces
    """
    interface = np.zeros(dgrid.shape).astype(np.bool_)
    [rows, cols] = dgrid.shape
    for i in range(rows):
        for j in range(cols):
            adj = ((i < rows - 1 and dgrid[i][j] * dgrid[i+1, j  ] <= 0) or
                   (i > 0 and dgrid[i][j] * dgrid[i-1, j  ] <= 0) or
                   (j < cols - 1 and dgrid[i][j] * dgrid[i,   j+1] <= 0) or
                   (j > 0 and dgrid[i][j] * dgrid[i,   j-1] <= 0))
            interface[i][j] = adj
    return interface

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(14, 5))

bubbles = dfun[50] >= 0  # vapor phase has non-negative distance.
liquid = dfun[50] < 0    # Liquid has negative distance.
interface = get_interface_mask(dfun[50])

data = {
    'Temperature': temp[50],
    'Velocity Mag.': mag[50],
    'Interface': interface,
    'Bubbles': bubbles,
    'Liquid': liquid
}

for idx, (key, im) in enumerate(data.items()):
    ax[idx].imshow(np.flipud(im))
    ax[idx].set_title(key)
    ax[idx].set_xticks([])
    ax[idx].set_yticks([])

### Accessing the metadata

There is a lot of metadata associated with each simulation. Pointers to some of the important metadata is listed in our [dataset documentation](https://github.com/HPCForge/BubbleML/blob/main/bubbleml_data/DOCS.md). The metadata is stored as a numpy array of tuples. Each tuple contains an array of bytes (the key) and a float (the value). The metadata stores critical information for the simulation. Some of these values (like the reynolds nunmber) will be necessary for training physics informed models.

In [ ]:
real_runtime_params = twall_100['real-runtime-params'][:]
key0, val0 = real_runtime_params[0]

print(f'Metadata size: {real_runtime_params.shape}')
print(f'Key type: {type(key0)}')
print(f'Val type: {type(val0)}')

def key_to_str(key):
    # convert byte string to a standard python utf-8 string.
    return key.decode('utf-8').strip()

# Convert to a dict of (string, float64)
runtime_param_dict = dict([(key_to_str(key), val) for (key, val) in real_runtime_params])

# print the reynolds number
inv_reynolds = runtime_param_dict['ins_invreynolds']
print(f'Reynolds Number: {1 / inv_reynolds}')

### Getting the Domain Size
The simulations in BubbleML are not all the same size. So, it can be beneficial to know how to access the true spatial dimensions. 
Here, we read the xy-extents from the metadata and set them as axis ticks on an image.

In [ ]:
xmin, xmax = runtime_param_dict['xmin'], runtime_param_dict['xmax']
ymin, ymax = runtime_param_dict['ymin'], runtime_param_dict['ymax']

print(f'x extents: {xmin}, {xmax}')
print(f'y extents: {ymin}, {ymax}')

plt.imshow(np.flipud(temp[50]), extent=[xmin,xmax,ymin,ymax])
plt.show()